# Homework 4
### Due Date:  Tuesday, September 26th at 11:59 PM

## Problem 1
The file `circles.txt` contains measurements of circle radii.  Your task is to write a script that reports the average area of the circles.  You will **not** use the `numpy` `mean` function.  Here are the requirements:
1. Open `circles.txt`, read in the data, and convert the data to floats.
2. Write a function that computes the area of a circle.
3. Write a function, called `myave`, that computes the average of the areas of the circles.  At the very least, `myave` should accept the list of radii as one argument and the circle function that you wrote in step 2 as another argument.  There are other ways of doing this task, but I want you to do it this way.
4. Print out the result.

In [2]:
# Read in circle radii and ensure each element is a float.
radii = open('circles.txt', 'r').read().splitlines()
radii = [float(c) for c in radii]
radii[:5]

[0.9681561892825645,
 0.9383571028606462,
 0.9633466725922948,
 1.1190000819885169,
 0.9015032470985564]

In [3]:
def circle_area(r, pi=3.1415926535):
    return pi * pow(r, 2)

def myave(circle_radii, f_area):
    """ Returns average area for circles represented by their radii in 
    circle_radii, and where f_area is a function to calculate a circle's 
    area by its radius.
    """
    return sum(f_area(radius) for radius in circle_radii) / len(circle_radii)

avg_area = myave(radii, circle_area)
print('Average circle area = {}'.format(avg_area))

Average circle area = 3.19589909699065


## Problem 2
The goal of this problem is to write a simple bank account withdraw system.  The problem is based off of one in _Structure and Interpretation of Computer Programs_.

**Instructions:** Do each part in a different cell block and clearly label each part.

### Part 1
Write a closure to make withdraws from a bank account.  The outer function should be accept the initial balance as an argument (I'll refer to this argument as `balance` in this problem statement, but you can call it whatever you want).  The inner function should accept the withdraw amount as an argument and return the new balance.

**NOTE1:** For this part, do not try to reassign `balance` in the inner function.  Just see what happens when you return a new balance.  You can store the new balance in a new name (call it `new_bal` if you want) or just return the new balance directly.

**NOTE2:** You may want to check for basic exceptions (e.g. attempting to withdraw more than the current balance).

Once you write your functions, demo them in your notebook as follows:
```python
wd = make_withdraw(init_balance)
wd(withdraw_amount)
wd(new_withdraw_amount)
```
You should observe that this does not behave correctly.  Why not?

### Part 2
You can fix things up by updating `balance` within the inner function.  But this won't work.  Try it out and explain why it doesn't work.  Try to use the language that we used in lecture.  **Hint:** [Python Execution Model](https://docs.python.org/3/reference/executionmodel.html).

### Part 3
Now, make just one small change to your code from Part 2.  Declare `balance` as a nonlocal variable using the nonlocal keyword.  That is, before you get to the inner function, say `nonlocal balance`.  Here's some information on the `nonlocal` statement:  [`nonlocal`](https://docs.python.org/3/reference/simple_stmts.html#nonlocal).

Now test things out like you did in Part 1.  It should be behaving correctly now.

### Part 4
Finally, visualize your code with [Python Tutor](http://pythontutor.com/) and embed your visualization in your notebook.  Pay attention to the variable `balance`.

In [4]:
# Part 1

def make_withdraw(balance):
    """ Returns callable. """
    def withdraw(amt):
        if amt > balance:
            raise ValueError('Withdrawal > Balance')
        
        return balance - amt
    return withdraw
        
wd = make_withdraw(100000)
wd(10000)
print('Balance after withdrawing 20,000: {}'.format(wd(10000)))

Balance after withdrawing 20,000: 90000


This does not work because we are only returning the new balance i.e. we are not updating the balance to a new balance anywhere in the code.

In [5]:
# Part 2

def make_withdraw_v2(balance):
    """ Returns callable. """
    def withdraw(amt):
        if amt > balance:
            raise ValueError('Withdrawal > Balance')
        # Set balance to new amount.
        balance -= amt
        return balance
    return withdraw

wd = make_withdraw_v2(100000)
wd(10000)
print('Balance after withdrawing 20,000: {}'.format(wd(10000)))

UnboundLocalError: local variable 'balance' referenced before assignment

This does not work either because once the argument `balance` is "captured" by the `withdraw` inner function i.e. once we return the inner function in:
    
    `return withdraw`

we have encapsulated the variable `balance` and prevented it from being changed. When we try updating the `balance` amount, we are effectively Whenever the function that is returned from `make_withdraw_v2` is called (which we store in the variable `wd`), at runtime it will decide that the `balance` variable is a local variable to the function (since we are assigning it an updated value within the function) and raise an error because we are referencing it before assigning anything to it.

## Problem 3
Let's return to the data from Problem 1.  Write two functions: 1.) The first function should return the average circle radius (you can re-use the one you already wrote if you'd like, but you might need to update it slightly for this problem) and 2.) The second function should just use `numpy` to compute the average.

Write a decorator to time the evaluation of each function.  You can use the timing decorator from lecture.

#### Notes and Hints
1. Be fair!
2. To be as fair as possible, do the following:
  1. Create an areas list/array _outside_ of your averaging functions.  This means that you should do a loop over the radii you read in from `circles.txt`, compute the area from each point, and store that area in an array.  Do you know why this is more fair?  Also, try to not use `append`.  Instead, preallocate space for your `area` list/array.
  2. Your `my_ave` function should accept your areas data as a list.  Remember, to allocate a list you should do `[0.0]*N`: if you use such a construct your list will will be filled in with zeros.
  3. Your `np_ave` function should accept your areas data as a `numpy` array.  To allocate a `numpy` array do `areas_np = np.zeros(len(radii))`.
  4. Now both functions are using the best data types possible for their tasks.

In [6]:
import time
import numpy as np

# Timer decorator from class.
def timer(f):
    def inner(*args):
        t0 = time.time()
        output = f(*args)
        elapsed = time.time() - t0
        print("Time Elapsed", elapsed)
        return output
    return inner

# Fill Python list of areas (by preallocating memory) and
# np array to enter into average functions.
list_area = [None] * len(radii)
areas_np = np.zeros(len(radii))
for i, radius in enumerate(radii):
    list_area[i] = circle_area(radius)
    areas_np[i] = circle_area(radius)
    
# Wrap numpy's and my average function with timer decorator.
my_ave = timer(lambda x: sum(x) / len(x))
np_ave = timer(lambda x: np.sum(x) / len(x))

# Call both functions to compare time.
my_ave(list_area)
np_ave(areas_np)

Time Elapsed 0.0
Time Elapsed 0.0


3.1958990969906487

## Problem 4
Write a decorator to check if a quantity returned from a function is positive.  An exception should be raised if the quantity is not positive.

Write three functions and decorate them with your decorator:
1. A function that returns the discriminant $\displaystyle d = b^{2} - 4ac$
2. A function that computes the absolute value (you must write this yourself...do not use built-ins)
3. A function of your own choosing.

Show that your decorator behaves correctly.  That is, for each function, show two cases (where applicable):
1. A case where positivity is not violated
2. A case where positivity is violated

In [12]:
def ensure_positive_result(f):
    """ Returns callable that raises RuntimeError if returned value is not positive. """
    def inner(*args):
        result = f(*args)
        if result < 0:
            raise RuntimeError('Returned value is negative')
        return result
    return inner

def discriminant(a, b, c):
    return pow(b, 2) - 4*a*c
    
def abs_custom(q):
    if q < 0:
        return -1*q
    return q

def years_to_tswift(ann_post_tax_salary):
    """ Returns number of years I'd have to work making annual (post-tax) salary to reach 
    Taylor Swift's net worth.
    """
    return 280000000 / ann_post_tax_salary
    
# Test discriminant function.
func_disc = ensure_positive_result(discriminant)
try: # Result is positive so function should work.
    discriminant = func_disc(1, 2, 1)
    print('Success: Discriminant function worked with following positive output: {}'.format(discriminant))
except:
    print('Failure: Discriminant function raised Exception when none was expected.')

try: # Should raise exception since result will be negative.
    discriminant = func_disc(2, 1, 2)
except Exception as e:
    print('Success: Discriminant function successfully raised error: {0}'.format(e))

# Test absolute value function, which under no circumstance should return a negative value.
func_abs = ensure_positive_result(abs_custom)
try:
    arg = -2
    result = func_abs(2)
    print('Success: Retrieved absolute value of {} with no exception: {}'.format(arg, result))
except:
    print('Failure: Raised exception retrieving absolute value when none expected.')
    
# Test custom function.
func_tswift = ensure_positive_result(years_to_tswift)
try:
    google_role_salary = 80000
    yrs = func_tswift(google_role_salary)
    print('Success: Computed positive yrs to reach T. Swift net worth off basic GOOG salary: {}'.format(yrs))
except:
    print('Failure: Raised exception retrieving func_tswift when none expected.')

try:
    spender_salary = -20000
    yrs = func_tswift(spender_salary)
    print('Failure: No exception raised when func_tswift() was negative.')
except:
    print('Success: Exception raised when func_tswift() was negative.')


Success: Discriminant function worked with following positive output: 0
Success: Discriminant function successfully raised error: Returned value is negative
Success: Retrieved absolute value of -2 with no exception: 2
Success: Computed positive yrs to reach T. Swift net worth off basic GOOG salary: 3500.0
Success: Exception raised when func_tswift() was negative.


## Problem 5
Coming soon...